In [10]:
import pandas as pd
import requests
from google.cloud import bigquery
from google.oauth2 import service_account
import datetime
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from tqdm import tqdm

load_dotenv()

GOOGLE_CREDENTIALS_PATH = os.getenv("GOOGLE_CREDENTIALS_PATH")
BQ_DATASET_ID = os.getenv("BQ_DATASET_ID")
BQ_TABLE_ID = os.getenv("BQ_TABLE_ID")



def load_to_table(df, dataset_id=BQ_DATASET_ID, table_id=BQ_TABLE_ID, credentials_path=GOOGLE_CREDENTIALS_PATH):
    """
    Uploads a pandas DataFrame to a Google BigQuery table.
    """
    try:
        credentials = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=["https://www.googleapis.com/auth/cloud-platform"]
        )
        client = bigquery.Client(credentials=credentials, project=credentials.project_id)

        table_id = f"{client.project}.{dataset_id}.{table_id}"

        job_config = bigquery.LoadJobConfig()

        job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
        job.result() 

        print(f"Data successfully loaded to table '{table_id}'.")

    except Exception as e:
        print(f"Error loading data to BigQuery: {e}")
        print(f"Failed to add data to table '{table_id}'.")


def get_libmev_data(start_date, end_date, interval_minutes=5):
    """
    Fetches data from the libMEV API within a specified date range.
    Parameters:
        start_date (int): Start timestamp in Unix epoch seconds.
        end_date (int): End timestamp in Unix epoch seconds.
        interval_minutes (int): Interval for API requests in minutes (default: 5 minutes)
    Returns: pd.DataFrame: A DataFrame containing the concatenated data from the API.
    """
    df_final = pd.DataFrame()
    interval_seconds = interval_minutes * 60
    total_intervals = (end_date - start_date) // interval_seconds


    for i in tqdm(range(total_intervals)):
        try:
            
            url = f"https://api.libmev.com/v1/bundles?timestampRange={end_date - interval_seconds},{end_date}"

            # Fetch and append data
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for HTTP issues

            data = response.json().get("data", [])
            if data:
                df = pd.DataFrame(data)
                df_final = pd.concat([df_final, df], ignore_index=True)

            # Update the end_date for the next iteration
            end_date -= interval_seconds

        except Exception as e:
            print(f"Unexpected error at timestamp {end_date}: {e}")

    # Filter, clean, and preprocess the data
    df_final =  df_final.loc[df_final.timestamp > start_date]
    df_final.index = pd.to_datetime(df_final.timestamp, unit= 's')
    df_final.drop('tokens', axis=1, inplace=True) 
    df_final.drop('token_balance_delta', axis=1, inplace=True) 
    df_final.rename_axis('DATETIME', inplace = True)

    print(f"DataFrame successfully created with {len(df_final)} rows.")
    return df_final


def get_BQ(query, credentials_path=GOOGLE_CREDENTIALS_PATH):
    """
    Fetches data from BigQuery using a SQL query.
    Parameters: query (str): The SQL query to execute.
    Returns: pd.DataFrame: The resulting DataFrame.
    """
    try:
        # Create a BigQuery engine using SQLAlchemy
        bq_engine = create_engine("bigquery://", credentials_path=credentials_path)
        df = pd.read_sql(query, bq_engine)
        return df
    except Exception as e:
        print(f"Error fetching data from BigQuery: {e}")
        return pd.DataFrame()


if __name__ == "__main__":

    start_timestamp = get_BQ(
        """
        SELECT MAX(TIMESTAMP) AS max_timestamp
        FROM `silken-mile-379810.libmev_dataset.searchers_bundles`
        """)
    
    start_timestamp = int(start_timestamp.iloc[0])
    
    end_timestamp = int(datetime.datetime.now().timestamp())  

    # Fetch data from libMEV
    df_libmev = get_libmev_data(start_timestamp, end_timestamp)

    # Load data into BigQuery
    load_to_table(df_libmev)

100%|██████████| 2065/2065 [16:02<00:00,  2.14it/s]


DataFrame successfully created with 32982 rows.
Data successfully loaded to table 'silken-mile-379810.libmev_dataset.searchers_bundles'.
